# 34. MNIST CNN with Ensemble

<p style="text-align: right;">
blackdew7@gmail.com<br>
Your name :
</p>

### 텐서플로 실습시간 그 일곱번째!<br>

#### 선행지식
1. TensorFlow 다루기 기초
2. 모델링을 한다는 것에 대한 이해.
3. Supervised Learning 중 Classification에 대한 기본 지식.
4. CNN에 대한 기본 구조와 개념

#### 실습목표
1. CNN의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
3. Dropout을 이해하고 사용할 수 있다.
4. Ensemble을 이해하고 사용할 수 있다.
5. CNN 레이어 구조를 Class로 구현할 수 있다.

#### 사용데이터.

01. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 라이브러리 불러오기

In [ ]:
# 메모리 초기화
%reset

# MNIST and Convolutional Neural Network
import numpy as np
import tensorflow as tf
import random

## 01. 데이터 불러오기 & 전처리

In [ ]:
# Graph Clear
tf.reset_default_graph()
tf.set_random_seed(2017) # random seeding - reproduct

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./resources/MNIST/", one_hot=True)

## 02. 제대로 복습해보자

In [ ]:
'''
지난 시간에 배운 Prepare Variables 코드를 완성해보자.

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

############################
# Place Holders

############################
# Weights & bias


In [ ]:
'''
지난 시간에 배운 CNN Graph 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

#########################
# ConvLayer 01 

#########################
# ConvLayer 02

#########################
# Dropout

#########################
# Fully Connected Layer

#########################
# Cost & Optimizer


In [ ]:
'''
지난 시간에 배운 Traning & Evaluation 코드를 완성해보자

여러분이 직접 스크립트를 짜야 합니다.
복사하여 붙여넣기를 하지 마시고, 직접 타이핑 하는 것을 권장합니다.
'''

###########################
# Initialize

###########################
# Training

###########################
# Evaluation


## 03. Class 만들기

In [ ]:
class ModelCNN:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._prepare_variables()
        self._build_net()
    
    def _prepare_variables(self):
        ############################
        # Place Holders

        self.X = tf.placeholder(tf.float32, [None, 784])
        self.X_img = tf.reshape(self.X, [-1, 28, 28, 1])
        self.Y = tf.placeholder(tf.float32, [None, 10])
        
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")

    def _build_net(self):
        #########################
        # ConvLayer 01 

        with tf.name_scope("layer1_conv"):

            # for Conv Layer 01 filter - shape=(3, 3, 1)
            W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01), name="W1")

            # Convolution Layer 01 -> (?, 28, 28, 32)
            L1_conv = tf.nn.conv2d(self.X_img, W1, strides=[1, 1, 1, 1], padding='SAME', name="L1_conv")
            L1_conv = tf.nn.relu(L1_conv, name="L1_relu")

            # Pooling Layer 01 -> (?, 14, 14, 32)
            L1_pool = tf.nn.max_pool(L1_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L1_max_pool")


        #########################
        # ConvLayer 02

        with tf.name_scope("layer2_conv"):

            # for Conv Layer 02 filter - shape=(3, 3, 32)
            W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01), name="W2")

            # Convolution Layer 02 -> (?, 14, 14, 32)
            L2_conv = tf.nn.conv2d(L1_pool, W2, strides=[1, 1, 1, 1], padding='SAME', name="L2_conv")
            L2_conv = tf.nn.relu(L2_conv, name="L2_relu")

            # Pooling Layer 02 -> (?, 7, 7, 64)
            L2_pool = tf.nn.max_pool(L2_conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name="L2_max_pool")


        #########################
        # Fully Connected Layer 01

        with tf.name_scope("layer1_fc"):

            ###############################
            # Flatten layer
            L_flat = tf.reshape(L2_pool, [-1, 7 * 7 * 64], name="L_flat")

            # for Final FC 7x7x64 inputs -> 1000
            W1_fc = tf.Variable(tf.random_normal([7 * 7 * 64, 1000], stddev=0.01), name="W1_fc")

            # for Final FC Layer 1: 7x7x64 inputs -> 1000
            L1_fc = tf.matmul(L_flat, W1_fc, name="L1_fc")
            L1_fc = tf.nn.relu(L1_fc, name="L1_fc_relu")

            #########################
            # Dropout
            L1_fc_dropout = tf.nn.dropout(L1_fc, keep_prob=self.keep_prob, name="dropout")
            #########################


        #########################
        # Fully Connected Layer 02

        with tf.name_scope("layer2_fc"):

            # for Final FC 1000 inputs -> 10
            W2_fc = tf.Variable(tf.random_normal([1000, 10], stddev=0.01), name="W2_fc")

            # bias
            b = tf.Variable(tf.random_normal([10]), name="bias")

            # for Final FC Layer 2: 1000 inputs -> 10
            self.logits = tf.add(tf.matmul(L1_fc_dropout, W2_fc), b, name="logits")


        #########################
        # Cost & Optimizer

        learning_rate = 0.001

        with tf.name_scope("Optimizer"):
            # Cost(loss) function & Optimizer
            cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y, name="cross_entropy")
            self.cost = tf.reduce_mean(cross_entropy, name="cost")
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)


        ###########################
        # Evaluation
        with tf.name_scope("Prediction"):
            # Test model and check accuracy
            correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1), name="compare")
            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

    def train(self, x_data, y_data):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: 0.7})
        
    def predict(self, x_test):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.keep_prob: 1.0})

    def get_accuracy(self, x_test, y_test):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: 1.0})

## 04. Ensemble

In [ ]:
# initialize
sess = tf.Session()

models = []
num_models = 5
for m in range(num_models):
    models.append(ModelCNN(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')


# train each model
for m_idx, m in enumerate(models):
    for i in range(200):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        c, _ = m.train(batch_xs, batch_ys)

        # 학습 상황 디스플레이
        if i % 10 == 0:
            print("%d modes, %dth records, training cost: %.3f" % ((m_idx + 1), (i * 100), c))

print('Learning Finished!')


In [ ]:
# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p
    
print(predictions)
print(tf.argmax(predictions, 1))

# ensemble accuracy
ensemble_correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(tf.cast(ensemble_correct_prediction, tf.float32))

print('Ensemble accuracy:', sess.run(ensemble_accuracy))

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [ ]:
# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
predictions = np.zeros([1, 10])
for m_idx, m in enumerate(models):
    p = m.predict(mnist.test.images[r:r + 1])
    predictions += p
    
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(tf.argmax(predictions, 1)))

import matplotlib.pyplot as plt
plt.imshow(mnist.test.images[r:r + 1].reshape(28, 28), cmap='Greys', interpolation='nearest')
plt.show()

In [ ]:
# 세션을 종료하자
sess.close()